# 2-2. Image Classification with GluonCV

## gluon-cv
<img src='https://cv.gluon.ai/_static/short_demo.gif' width=100% />

### install
- [참고](https://cv.gluon.ai/contents.html#installation)

### cpu only version install

In [ ]:
# cpu only
# !pip3 install --upgrade mxnet > /dev/null

### gpu version install
- deep learning library 버전을 확인한다

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


### cuda version에 따라 mxnet-cuXXX를 설치한다
- 10.0 : `mxnet-cu100`
- 10.1 : `mxnet-cu101`
- 10.2 : `mxnet-cu102`

In [ ]:
# CUDA 버전이 10.1인경우 mxnet-cu101을 설치한다
!pip3 install --upgrade mxnet-cu101

     |███████████████████████████████▌| 834.1MB 1.3MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x39d92000 @  0x7efc8bc00615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd
     |████████████████████████████████| 846.0MB 21kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


### gluon-cv install

In [ ]:
!pip3 install --upgrade gluoncv

     |████████████████████████████████| 1.0MB 8.3MB/s 
     |████████████████████████████████| 11.8MB 10.6MB/s 
     |████████████████████████████████| 317kB 37.1MB/s 
     |████████████████████████████████| 256kB 51.1MB/s 
     |████████████████████████████████| 675kB 49.9MB/s 
     |████████████████████████████████| 133kB 41.8MB/s 
     |████████████████████████████████| 972kB 41.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 102kB 9.2MB/s 
     |████████████████████████████████| 215kB 56.5MB/s 
     |████████████████████████████████| 7.2MB 24.4MB/s 
     |████████████████████████████████| 71kB 12.1MB/s 
     |████████████████████████████████| 2.6MB 55.9MB/s 
     |████████████████████████████████| 962kB 20.6MB/s 
     |████████████████████████████████| 71kB 6.4MB/s 
     |████████████████████████████████| 102kB 9.4MB/s 
  Created wheel for ConfigSpace: filena

## image classification

## 이미지 준비

In [ ]:
!curl https://raw.githubusercontent.com/dmlc/web-data/master/gluoncv/classification/mt_baker.jpg > mt_baker.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  304k  100  304k    0     0   905k      0 --:--:-- --:--:-- --:--:--  905k


<img src='https://raw.githubusercontent.com/dmlc/web-data/master/gluoncv/classification/mt_baker.jpg' height=300px/>

### 라이브러리 import

In [ ]:
from mxnet import nd, image

import gluoncv as gcv
gcv.utils.check_version('0.6.0')
from gluoncv.data.transforms.presets.imagenet import transform_eval
from gluoncv.model_zoo import get_model

### 분류 모델 loading
- modelzoo에서 모델을 다운받아온다

In [ ]:
model_name = 'ResNet50_v2'

In [ ]:
net = get_model(model_name, pretrained=True)

92863KB [00:01, 72033.62KB/s]                           


### 결과값의 유형

In [ ]:
classes = net.classes

In [ ]:
len(classes)

1000

### image 변환

In [ ]:
image_file='./mt_baker.jpg'

In [ ]:
img = image.imread(image_file)

In [ ]:
type(img)

mxnet.ndarray.ndarray.NDArray

In [ ]:
img.shape

(1458, 3000, 3)

### resnet model 에 넣는 데이터 형태로 변환
- 224*224

### QUIZ. 

왜 244*244로 변환하는건가요?

### imagenet transform 함수 활용

In [ ]:
img = transform_eval(img)

In [ ]:
type(img)

mxnet.ndarray.ndarray.NDArray

In [ ]:
img.shape

(1, 3, 224, 224)

### 분류모델 실행

In [ ]:
pred = net(img)

### 분류결과 출력

In [ ]:
topK = 5
ind = nd.topk(pred, k=topK)[0].astype('int')
print('The input picture is classified to be')
for i in range(topK):
    print('\t[%s], with probability %.3f.'%
          (classes[ind[i].asscalar()], nd.softmax(pred)[0][ind[i]].asscalar()))

The input picture is classified to be
	[volcano], with probability 0.530.
	[alp], with probability 0.465.
	[valley], with probability 0.002.
	[mountain tent], with probability 0.001.
	[lakeside], with probability 0.000.


# 함수로 만들어서 간단하게 활용하자

In [ ]:
def get_classify_result(model_name, image_file):
  # model load
  net = get_model(model_name, pretrained=True)
  classes = net.classes

  # image load
  img = image.imread(image_file)
  img = transform_eval(img)

  # predict(classify)
  pred = net(img)

  topK = 5
  ind = nd.topk(pred, k=topK)[0].astype('int')
  print('The input picture is classified to be')
  for i in range(topK):
      print('\t[%s], with probability %.3f.'%
            (classes[ind[i].asscalar()], nd.softmax(pred)[0][ind[i]].asscalar()))

In [ ]:
get_classify_result( 'ResNet50_v2','./mt_baker.jpg' )

The input picture is classified to be
	[volcano], with probability 0.530.
	[alp], with probability 0.465.
	[valley], with probability 0.002.
	[mountain tent], with probability 0.001.
	[lakeside], with probability 0.000.


# Transfer Learning
- [Resnet기반의 이미지 분류기 만들기](https://cv.gluon.ai/build/examples_classification/transfer_learning_minc.html)
- 연예인사진 유사도 같은 프로그램 이렇게 만들 수 있습니다
- 회사 연말행사에 닮은사람 이벤트 하면 재미있겠네요

## image download
- 실습을 위해 작은 파일셋을 내려받아서 진행


In [31]:
import zipfile, os
from gluoncv.utils import download

file_url = 'https://raw.githubusercontent.com/dmlc/web-data/master/gluoncv/classification/minc-2500-tiny.zip'
zip_file = download(file_url, path='./')
with zipfile.ZipFile(zip_file, 'r') as zin:
    zin.extractall(os.path.expanduser('./'))

8038KB [00:00, 28501.25KB/s]                          


## 학습에 필요한 라이브러리

In [32]:
import mxnet as mx
import numpy as np
import os, time, shutil

from mxnet import gluon, image, init, nd
from mxnet import autograd as ag
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms
from gluoncv.utils import makedirs
from gluoncv.model_zoo import get_model

## 학습(Train) 변수

In [33]:
classes = 23

epochs = 5
lr = 0.001
per_device_batch_size = 1
momentum = 0.9
wd = 0.0001

lr_factor = 0.75
lr_steps = [10, 20, 30, np.inf]

num_gpus = 0
num_workers = 8
ctx = [mx.gpu(i) for i in range(num_gpus)] if num_gpus > 0 else [mx.cpu()]
batch_size = per_device_batch_size * max(num_gpus, 1)

## 데이터 증폭(Augmentation)
- Randomly crop the image and resize it to 224x224
- Randomly flip the image horizontally
- Randomly jitter color and add noise
- Transpose the data from height*width*num_channels to num_channels*height*width, and map values from [0, 255] to [0, 1] Normalize with the mean and standard deviation from the ImageNet dataset.

In [34]:
jitter_param = 0.4
lighting_param = 0.1

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomFlipLeftRight(),
    transforms.RandomColorJitter(brightness=jitter_param, contrast=jitter_param,
                                 saturation=jitter_param),
    transforms.RandomLighting(lighting_param),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

## data load/pre processing

In [35]:
path = './minc-2500-tiny'
train_path = os.path.join(path, 'train')
val_path = os.path.join(path, 'val')
test_path = os.path.join(path, 'test')

train_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(train_path).transform_first(transform_train),
    batch_size=batch_size, shuffle=True, num_workers=num_workers)

val_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(val_path).transform_first(transform_test),
    batch_size=batch_size, shuffle=False, num_workers = num_workers)

test_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(test_path).transform_first(transform_test),
    batch_size=batch_size, shuffle=False, num_workers = num_workers)

## 기존 학습된 모델 로딩

In [36]:
model_name = 'ResNet50_v2'
finetune_net = get_model(model_name, pretrained=True)
# output 재정의
with finetune_net.name_scope():
    finetune_net.output = nn.Dense(classes)
finetune_net.output.initialize(init.Xavier(), ctx = ctx)
finetune_net.collect_params().reset_ctx(ctx)
finetune_net.hybridize()

trainer = gluon.Trainer(finetune_net.collect_params(), 'sgd', {
                        'learning_rate': lr, 'momentum': momentum, 'wd': wd})
metric = mx.metric.Accuracy()
L = gluon.loss.SoftmaxCrossEntropyLoss()

## 학습도중의 validation과 test를 위해서 test function 정의

In [37]:
def test(net, val_data, ctx):
    metric = mx.metric.Accuracy()
    for i, batch in enumerate(val_data):
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
        outputs = [net(X) for X in data]
        metric.update(label, outputs)

    return metric.get()

## train

In [38]:
lr_counter = 0
num_batch = len(train_data)

for epoch in range(epochs):
    if epoch == lr_steps[lr_counter]:
        trainer.set_learning_rate(trainer.learning_rate*lr_factor)
        lr_counter += 1

    tic = time.time()
    train_loss = 0
    metric.reset()

    for i, batch in enumerate(train_data):
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
        with ag.record():
            outputs = [finetune_net(X) for X in data]
            loss = [L(yhat, y) for yhat, y in zip(outputs, label)]
        for l in loss:
            l.backward()

        trainer.step(batch_size)
        train_loss += sum([l.mean().asscalar() for l in loss]) / len(loss)

        metric.update(label, outputs)

    _, train_acc = metric.get()
    train_loss /= num_batch

    _, val_acc = test(finetune_net, val_data, ctx)

    print('[Epoch %d] Train-acc: %.3f, loss: %.3f | Val-acc: %.3f | time: %.1f' %
             (epoch, train_acc, train_loss, val_acc, time.time() - tic))

_, test_acc = test(finetune_net, test_data, ctx)
print('[Finished] Test-acc: %.3f' % (test_acc))

[Epoch 0] Train-acc: 0.017, loss: 4.111 | Val-acc: 0.087 | time: 69.7
[Epoch 1] Train-acc: 0.035, loss: 4.096 | Val-acc: 0.043 | time: 69.1
[Epoch 2] Train-acc: 0.026, loss: 4.355 | Val-acc: 0.065 | time: 69.5
[Epoch 3] Train-acc: 0.000, loss: 4.053 | Val-acc: 0.000 | time: 69.2
[Epoch 4] Train-acc: 0.043, loss: 3.924 | Val-acc: 0.043 | time: 69.2
[Finished] Test-acc: 0.065


# 실습


## 1. 이미지넷 모델 중에서 volcano 정확도가 가장 높은 모델을 찾기
- [ImageNet](https://cv.gluon.ai/model_zoo/classification.html#imagenet)

In [24]:
get_classify_result( 'ResNet34_v1','./mt_baker.jpg' )

  0%|          | 0/79094 [00:00<?, ?KB/s]

79095KB [00:01, 63757.20KB/s]                           


The input picture is classified to be
	[volcano], with probability 0.985.
	[alp], with probability 0.013.
	[valley], with probability 0.001.
	[mountain tent], with probability 0.000.
	[lakeside], with probability 0.000.


## 2. 아래의 이미지를 이용하여 비행기 정확도가 가장 높은 모델을 찾기


<img src='https://previews.123rf.com/images/akinshin/akinshin1508/akinshin150800008/43579353-%ED%9D%B0%EC%83%89-%EB%B0%B0%EA%B2%BD-%EB%B2%A1%ED%84%B0-%EC%9D%BC%EB%9F%AC%EC%8A%A4%ED%8A%B8-%EB%A0%88%EC%9D%B4-%EC%85%98%EC%97%90-%EB%B9%84%ED%96%89%EA%B8%B0-%EC%8B%A4%EB%A3%A8%EC%97%A3.jpg' height=300px/>

In [25]:
!curl https://previews.123rf.com/images/akinshin/akinshin1508/akinshin150800008/43579353-%ED%9D%B0%EC%83%89-%EB%B0%B0%EA%B2%BD-%EB%B2%A1%ED%84%B0-%EC%9D%BC%EB%9F%AC%EC%8A%A4%ED%8A%B8-%EB%A0%88%EC%9D%B4-%EC%85%98%EC%97%90-%EB%B9%84%ED%96%89%EA%B8%B0-%EC%8B%A4%EB%A3%A8%EC%97%A3.jpg > plane.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 38625  100 38625    0     0   275k      0 --:--:-- --:--:-- --:--:--  275k


In [30]:
get_classify_result( 'DenseNet201','./plane.jpg' )

74804KB [00:01, 44822.47KB/s]                           


The input picture is classified to be
	[warplane], with probability 0.856.
	[wing], with probability 0.054.
	[airliner], with probability 0.045.
	[aircraft carrier], with probability 0.012.
	[projectile], with probability 0.005.
